In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
import numpy as np

#Data
def generate_data(num_samples, max_seq_length, vocab_size):
    X = np.random.randint(1, vocab_size, (num_samples, max_seq_length))
    y = np.random.randint(1, vocab_size, (num_samples, max_seq_length))
    return X, y

num_samples = 10000
max_seq_length = 10
vocab_size = 50

encoder_input_data, decoder_input_data = generate_data(num_samples, max_seq_length, vocab_size)
decoder_target_data = np.roll(decoder_input_data, shift=-1, axis=1)
decoder_target_data[:, -1] = 0

#Seq2Seq Model
latent_dim = 256

#encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=vocab_size, output_dim=latent_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

#decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=vocab_size, output_dim=latent_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

#Compile
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64,
          epochs=10,
          validation_split=0.2)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 256)            12800     ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 256)            12800     ['input_2[0][0]']             
                                                                                              

In [2]:
#Inference Setup

# Encoder
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_lstm_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

#Function to Decode Sequences
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = 1


    decoded_sentence = []
    stop_condition = False
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        decoded_sentence.append(sampled_token_index)
        if sampled_token_index == 0 or len(decoded_sentence) > max_seq_length:
            stop_condition = True
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]
    return decoded_sentence

In [3]:
#Test
print("encoded sentence:",encoder_input_data[0:1])
test_input = encoder_input_data[0:1]
decoded_sentence = decode_sequence(test_input)
print('Decoded sentence:', decoded_sentence)

encoded sentence: [[33 17 26 42 38 15 33  6 35 26]]
1/1 [==============================] - 0s 41ms/step
Decoded sentence: [35, 35, 47, 10, 22, 34, 22, 34, 38, 0]
